In [119]:
import cv2
import time
from ppadb.client import Client
import math
import numpy as np
from scipy.spatial import distance as dist

In [120]:
adb = Client(host='127.0.0.1', port=5037)
devices = adb.devices()

if len(devices) == 0:
    print('no device attached')
    quit()

device = devices[0]
  
def convert_to_cv2_img(image):
    
    with open('screen.png', 'wb') as f:
        f.write(image)
        
    img = cv2.imread("screen.png")
    
    f = int(img.shape[0] * 0.2)
    
    # crop image to remove 40% from top and 20% from bottom of the image
    img = img[f*2:-f, 0:]
    
    # resize image to a fixed size
    img = cv2.resize(img, (1080, 1960))
    
    return img

In [133]:
def get_centroid(c):
    
    M = cv2.moments(c)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    return [cX, cY]


def find_starting_pillar(d):
    return min(d.items(), key=lambda x:x[1]['centroid'][0])

def find_ending_pillar(d):
    return max(d.items(), key=lambda x:x[1]['centroid'][0])

def get_distance_to_move(contours):
    
    contours_filtered = sorted(contours, key=lambda x:cv2.contourArea(x), reverse=True)[:2]  
    
    d = dict()
    for idx, c in enumerate(contours_filtered):
        
        x,y,w,h = cv2.boundingRect(c)
        centroid = get_centroid(c)
        
        d[idx] = {
            "edges" : [x, y, w, h],
            "centroid": centroid
        }

    start = find_starting_pillar(d)[1]
    end = find_ending_pillar(d)[1]
    
    start_xy = start.get("centroid")
    end_xy = end.get("centroid")
    
    x,y,w,h = start.get("edges")
    start_xy[0] = x + w
    start_xy[1] = end_xy[1]
            
    # find distance as euclidiean distance and round adjust it by 0.98 multiplication and floor
    distance = int(math.floor(dist.euclidean(start_xy, end_xy)) * 0.98)
    
    return distance
       

def play():
    
    # capture screenshot
    cap = device.screencap()
    
    # convert image to opencv format and then to HSV
    img = convert_to_cv2_img(cap)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # range for black -- color of pillars
    l_range_low = np.array([0, 0, 0])
    l_range_high = np.array([180, 255, 30])
    
    # threshold and convert image to binary
    img_bin = cv2.inRange(img_hsv, l_range_low, l_range_high)
    
    # find all countours
    contours, hierarchy = cv2.findContours(img_bin,  
                    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    
    # helper function to get distance
    distance = adjust_starting_pos(contours)
        
    # give a touch input for distance milli seconds at any point in the screen
    device.shell(f'input touchscreen swipe 500 500 500 500 {distance}')
    
    return distance

In [134]:
for i in range(100):
    print(f"Jump no : {i+1}")
    distance = play()
    print(f"Distance : {distance}")
    time.sleep(3)

Jump no : 1
Distance : 714
Jump no : 2
Distance : 555
Jump no : 3
Distance : 701
Jump no : 4
Distance : 683
Jump no : 5
Distance : 655
Jump no : 6
Distance : 424
Jump no : 7
Distance : 758
Jump no : 8
Distance : 596
Jump no : 9
Distance : 493
Jump no : 10
Distance : 858
Jump no : 11
Distance : 388
Jump no : 12
Distance : 316
Jump no : 13
Distance : 594
Jump no : 14
Distance : 280
Jump no : 15
Distance : 219
Jump no : 16
Distance : 394
Jump no : 17
Distance : 625
Jump no : 18
Distance : 732
Jump no : 19
Distance : 472
Jump no : 20
Distance : 718
Jump no : 21
Distance : 735
Jump no : 22
Distance : 679
Jump no : 23
Distance : 279
Jump no : 24
Distance : 371
Jump no : 25
Distance : 760
Jump no : 26
Distance : 747
Jump no : 27
Distance : 324
Jump no : 28
Distance : 484
Jump no : 29
Distance : 541
Jump no : 30
Distance : 239
Jump no : 31
Distance : 411
Jump no : 32
Distance : 211
Jump no : 33
Distance : 548
Jump no : 34
Distance : 578
Jump no : 35
Distance : 436
Jump no : 36
Distance : 168
J